In [ ]:
import numpy as np
import tqdm
from scipy import misc
import pandas as pd
import os.path
import matplotlib.pyplot as plt, matplotlib as mpl
%matplotlib inline

In [ ]:
import loadData, os

In [ ]:
from importlib import reload
reload(loadData)

In [ ]:
ls /home/tsbertalan/data2/behavioralCloning/*.zip

In [ ]:
# dataGenerator = loadData.DataGenerator(
#     [
#         os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
#         for p in (
#             'data_provided.zip',
# #             'mouseForward.zip',
# #             'mouseReverse.zip',
#         )
#     ],
#     responseKeys=('steering',),# 'throttle', 'brake',)
# )
# gen = dataGenerator.generate(stopOnEnd=True, epochSubsampling=.2)
# XY = [xy for xy in tqdm.tqdm_notebook(gen)]
# X = np.vstack([x for (x, y) in XY])
# Y = np.vstack([y for (x, y) in XY])
# X.shape, Y.shape

In [ ]:
# del X, Y, XY, dataGenerator
import gc
gc.collect()

In [ ]:
dataGenerator = loadData.CenterOnlyDataGenerator(
    [
        os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
        for p in (
            'data_provided.zip',
            'mouseForward.zip',
            'mouseReverse.zip',
        )
    ],
)
gen = dataGenerator.generate(stopOnEnd=True, epochSubsampling=.2)
Y = Yunbalanced = np.vstack([y for (x, y) in tqdm.tqdm_notebook(gen)])

In [ ]:
binWidth = 2* min(np.abs(Y[Y!=0]))
edges = []
x = binWidth / 2
while x < max(Y):
    edges.append(x)
    x += binWidth
edges.append(x)

x = -binWidth / 2
while x > min(Y):
    edges.append(x)
    x -= binWidth
edges.append(x)

edges.sort()

len(edges)

In [ ]:
hist, bin_edges = np.histogram(Y, bins='auto')
bin_edges.shape, max(hist[hist != max(hist)]) / max(hist), len(Y[Y==0]), len(Y[Y!=0])

In [ ]:
Y = Yunbalanced
fig, ax = plt.subplots()
kw = dict(bins=64, alpha=.25, normed=True)
ax.hist(Y, label='full distribution',color='red', **kw)
ax.hist(Y[Y!=0], label='nonzero only', color='blue', **kw)
# ax.set_xlim(-.25, .25)
ax.legend()
ax.set_xlabel('turn angle')
ax.set_ylabel('normed count');

In [ ]:
# This way produces two big spikes for the left/right cameras associated with the zero-angle rows.

# class DeemphasizedZeroDataGenerator(loadData.DataGenerator):
#     zeroKeepProbability = .07
#     sideCameraChance = .6
    
#     rowsPerSample = 1
#     def sample(self, validation=False):
#         # Could be really inefficient; we read 3*n images, where n depends on the number of failed draws!
#         # In practice, though, we don't seem to fail too much.
#         p = lambda: random.uniform(0, 1)
#         while True:
#             X, Y = super().sample(validation=validation)
#             if p() < self.sideCameraChance:
#                 i = random.choice([0, 2])
#                 x, y = X[i], Y[i]
#                 break
#             elif p() < self.zeroKeepProbability:
#                 x, y = X[1], Y[1]
#                 break
#         return x.reshape((1, *x.shape)), y

In [ ]:
import random
_LCR = ('left', 'center', 'right')

class DeemphasizedZeroDataGenerator(loadData.DataGenerator):
    zeroKeepProbability = .07
    
    def sampleRow(self, validation=False):

        while True:
            # Get the index and increment the state.
            indices = self._indices[validation]
            state = self._state[validation]
            j = indices[state]

            # Reset or increment the counter.
            if self._state[validation] == len(indices) - 1:
                self._state[validation] = 0
            else:
                self._state[validation] += 1

            # Get the left, center, and right image paths
            # and corresponding requested response variables.
            lcrImagePaths = [self.log.at[j, key] for key in _LCR]
            response = np.array([self.log.at[j, key] for key in self.responseKeys])
            
            # Check for acceptable steering angle.
            if response[0] != 0 or random.uniform(0, 1) < self.zeroKeepProbability:
                break
            
        return lcrImagePaths, response

In [ ]:
datadir = os.path.join(loadData.HOME, 'data2', 'behavioralCloning')

In [ ]:
dataGenerator = DeemphasizedZeroDataGenerator(
    os.path.join(datadir, 'data_provided.zip')
)

In [ ]:
Y = Ybalanced = np.vstack([dataGenerator.sample()[1] for _ in tqdm.tqdm_notebook(range(3000))])
Y[Y==0].shape

In [ ]:
fig, ax = plt.subplots()
kw = dict(bins=64, alpha=.25, normed=True)
ubN, ubBins, ubPatches = ax.hist(Yunbalanced, label='full distribution', color='red', **kw)
ubPatches[0].set_label('full distribution\n(max normed count is %.3g)' % max(ubN))
ax.hist(Ybalanced, label='zero-centered kept w.p. %s' % dataGenerator.zeroKeepProbability, color='blue', **kw)
# ax.set_xlim(-.25, .25)
# ax.set_ylim(0, 4)
ax.legend(fontsize=12, loc='right')
ax.set_xlabel('turn angle [rad]')
ax.set_ylabel('normed count');
fig.savefig('doc/zeroCenteredKeptWP%s.png' % str(dataGenerator.zeroKeepProbability).replace('.', 'p'))

In [ ]:
epochs = 10

In [ ]:
import keras
import models, nnUtils
Model = models.Nvidia

In [ ]:
%%time
modelname = 'deemphasizedZeroLR-mouse-kl2p0001'
kernel_regularizer = keras.regularizers.l2(0.0001)
#bias_regularizer = keras.regularizers.l2(0.01)

model = modelReg = Model(
    1, dataGenerator.sampleImageShape,
    optimizer=keras.optimizers.Nadam(
        lr=0.0001, #beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    ),
    kernel_regularizer=kernel_regularizer,
    #bias_regularizer=bias_regularizer,
)

trainGen = dataGenerator.generate(epochSubsampling=1.0)
validGen = dataGenerator.generate(validation=True)

history = nnUtils.fitModelWithDataGenerator(model, dataGenerator, modelname, epochs=epochs)

fpath = os.path.join(datadir, '%s.h5' % modelname)
print('Saving to', fpath)
model.save(fpath)

trainIndices, validationIndices = dataGenerator._indices
for ind, lab in zip(dataGenerator._indices, ('train', 'validation')):
    
    imgs = []
    responses = []
    for k in range(ind[0], ind[0]+1000):
        try:
            image, response = dataGenerator.getImageResponse(k)
            imgs.append(image.reshape((1, *image.shape)))
            responses.append(response)
        except KeyError:
            break
    X = np.vstack(imgs)
    Y = np.vstack(responses)

    pred = model.predict(X)
    pred.shape, Y.shape

    from collections import deque
    def runningMean(x, N):
        y = []
        hist = deque(maxlen=N)
        for a in x:
            hist.append(a)
            y.append(np.mean(hist))
        return np.array(y)
    
    ind = np.copy(ind)
    ind.sort()
    fig, ax = plt.subplots()
    start = 0
    end = start + 200
    ax.plot(Y[start:end], label=r'truth $\theta$', color='black')
    ax.plot(pred[start:end], label=r'predictions $\hat\theta$', color='red')
    filtersize = 3
    scale = 1
    smoothed = scale * runningMean(pred, filtersize)
    ax.plot(
        smoothed[start:end], 
        label=r'$\hat\rho = %.1f \cdot box_{%d}(\hat\theta)$' % (scale, filtersize),
        alpha=.25,
        color='red',
    )
    ax.set_xlabel('time [samples]')
    ax.set_ylabel('angle [radians]')
    ax.set_title(lab)
    ax.legend();
    fig.savefig('doc/smoothingEffect-%s-%s.png' % (modelname, lab))